Họ và tên: **Nguyễn Ngọc Băng Tâm**

MSSV: **1712747**

In [6]:
import os

from requests_html import HTMLSession
import requests
import pandas as pd
import time
import re

import urllib.robotparser # Kiểm tra file robot.txt có được phép crawl không

In [7]:
if not os.path.exists('Api_data/'):
    os.makedirs('Api_data/')
    
if not os.path.exists('Crawl_data/'):
    os.makedirs('Crawl_data/')

## 1. API

Test dữ liệu của một user

In [8]:
url = 'https://api-v2.soundcloud.com/users/2?client_id=koSuixzk25Cz0fDgsPfhkgeZcKBr7lpQ'

In [9]:
r = requests.get(url)
r.text

'{"avatar_url":"https://i1.sndcdn.com/avatars-LQHwEEFP8SciWWEl-yGLHbw-large.jpg","city":"Berlin","comments_count":383,"country_code":"DE","created_at":"2007-07-20T16:07:00Z","creator_subscriptions":[{"product":{"id":"creator-pro-unlimited"}}],"creator_subscription":{"product":{"id":"creator-pro-unlimited"}},"description":"Founder & CEO Dance. Founder SoundCloud and artist http://soundcloud.com/forss","followers_count":45999,"followings_count":2000,"first_name":"Eric","full_name":"Eric Quidenus-Wahlforss","groups_count":0,"id":2,"kind":"user","last_modified":"2020-07-21T08:17:59Z","last_name":"Quidenus-Wahlforss","likes_count":2582,"playlist_likes_count":392,"permalink":"eric","permalink_url":"https://soundcloud.com/eric","playlist_count":24,"reposts_count":null,"track_count":64,"uri":"https://api.soundcloud.com/users/2","urn":"soundcloud:users:2","username":"Eric 🔥","verified":false,"visuals":{"urn":"soundcloud:users:2","enabled":true,"visuals":[{"urn":"soundcloud:visuals:105136722","e

In [10]:
r.headers

{'Content-Type': 'application/json; charset=utf-8', 'Content-Length': '585', 'Connection': 'keep-alive', 'Date': 'Mon, 02 Nov 2020 11:10:03 GMT', 'X-Robots-Tag': 'noindex', 'Cache-Control': 'private, max-age=0', 'Referrer-Policy': 'no-referrer', 'X-Frame-Options': 'DENY', 'X-Content-Type-Options': 'nosniff', 'Content-Encoding': 'gzip', 'Strict-Transport-Security': 'max-age=63072000', 'Server': 'am/2', 'Vary': 'Origin', 'X-Cache': 'Miss from cloudfront', 'Via': '1.1 fdf96859c8a26606c28f770011e1fdeb.cloudfront.net (CloudFront)', 'X-Amz-Cf-Pop': 'HKG60-C1', 'X-Amz-Cf-Id': 'oSxfO9t4S0mB6CWKaWEqc_2NX-mE7sGYw1F8-jmtGGISwQ4BOKUCTg=='}

In [11]:
json_pydata = r.json()
json_pydata

{'avatar_url': 'https://i1.sndcdn.com/avatars-LQHwEEFP8SciWWEl-yGLHbw-large.jpg',
 'city': 'Berlin',
 'comments_count': 383,
 'country_code': 'DE',
 'created_at': '2007-07-20T16:07:00Z',
 'creator_subscriptions': [{'product': {'id': 'creator-pro-unlimited'}}],
 'creator_subscription': {'product': {'id': 'creator-pro-unlimited'}},
 'description': 'Founder & CEO Dance. Founder SoundCloud and artist http://soundcloud.com/forss',
 'followers_count': 45999,
 'followings_count': 2000,
 'first_name': 'Eric',
 'full_name': 'Eric Quidenus-Wahlforss',
 'groups_count': 0,
 'id': 2,
 'kind': 'user',
 'last_modified': '2020-07-21T08:17:59Z',
 'last_name': 'Quidenus-Wahlforss',
 'likes_count': 2582,
 'playlist_likes_count': 392,
 'permalink': 'eric',
 'permalink_url': 'https://soundcloud.com/eric',
 'playlist_count': 24,
 'reposts_count': None,
 'track_count': 64,
 'uri': 'https://api.soundcloud.com/users/2',
 'urn': 'soundcloud:users:2',
 'username': 'Eric 🔥',
 'verified': False,
 'visuals': {'urn'

### Định nghĩa hàm **collect_data_api** để crawl dữ liệu từ api của Soundcloud
- Check rate limit: https://developers.soundcloud.com/docs/api/rate-limits#errors
- **Lưu ý:**
    - Api của Soundcloud đã được thay đổi thành version 2 (thêm từ khóa -v2) và cấu trúc file json thu được từ api cũng có thay đổi so với tài liệu trên trang chủ.
    - Trong api lấy dữ liệu có tham số cần truyền vào là `client-id`. Do Soundcloud không còn cho phép đăng ký `client-id` để thực hiện crawl data nên ta sử dụng cách thức bắt và xem gói tin trong tab Network (F12) để lấy trường dữ liệu này. 

In [79]:
def collect_data_api(data_type, client_id, num_of_samples, output_file, sleep_time=1):
    """
    - data_type: types of data to collect, one out of three types {users, playlists, tracks}
    - client_id: registered id to get access to public data
    - num_of_samples: number of samples to fetch
    - output_file: name of output file
    - sleep_time: time to sleep to avoid limit rate
    """
    
    json_lst = []
   
    data_id = 1
    offset = 0
        
    # provide headers to avoid 403 Too many request status code
    headers = {'User-Agent': 'Mozilla/5.0'}
    
    step = data_id
    while (data_id < num_of_samples + offset + 1):
        url = f'https://api-v2.soundcloud.com/{data_type}/{step}?client_id={client_id}'
        print(url)
        r = requests.get(url, headers=headers)
        
        if r.ok == True:
            json_pydata = r.json()
            json_lst.append(json_pydata)
        
        else:
            time.sleep(sleep_time)
            # when data_api is invalid with <data_id> 
            # increase an offset of 1 to ensure the total amount of data is <num_of_samples>
            offset += 1
            
        data_id += 1
        step += 4
            
    df = pd.json_normalize(json_lst)
    df = df.reset_index()
    df.to_csv(output_file, sep='\t', index=False, header=True, encoding='utf-8')

In [80]:
sample_size = 1000

### API - users

In [ ]:
# TEST with users
collect_data_api('users', 'koSuixzk25Cz0fDgsPfhkgeZcKBr7lpQ', sample_size, 'Api_data/user.csv')
users = pd.read_csv('Api_data/user.csv', sep='\t')
assert users.shape[0] == 1000

### API - playlists

In [ ]:
# TEST with playlists
collect_data_api('playlists', 'koSuixzk25Cz0fDgsPfhkgeZcKBr7lpQ', sample_size, 'Api_data/playlist.csv')
playlists = pd.read_csv('Api_data/playlist.csv', sep='\t')
assert playlists.shape[0] == 1000

### API - tracks - ON-GOING

In [ ]:
# TEST with tracks
collect_data_api('tracks', 'yLDMxXMZsSiBSpUG5psrlryn56TID1wS', sample_size, 'Api_data/track.csv')
# collect_data_api('tracks', 'koSuixzk25Cz0fDgsPfhkgeZcKBr7lpQ', sample_size, 'Api_data/track.csv')
tracks = pd.read_csv('Api_data/track.csv', sep='\t')
assert tracks.shape[0] == 1000

### Tham khảo

- [Cách lấy client-id Soundcloud](https://stackoverflow.com/questions/54753238/soundcloud-application-registration-form-is-closed)